## Tratamento da tabela 'patient01'
Neste caderno será apresentada a exploração inicial e tratamento dos dados da tabela 'patient01'.

### Importação das bibliotecas
 
A seguir uma breve apresentação de cada biblioteca que será utilizada:

- A biblioteca [NumPy](https://numpy.org/) é fundamental para qualquer tipo de computação científica em Python
- A biblioteca [pandas](https://pandas.pydata.org/) é a nossa ferramenta pricipal para análise e manipulação de dados
- A biblioteca [python-decouple](https://github.com/henriquebastos/python-decouple) auxiliar para trabalhar com variaveis de ambiente
- A biblioteca [SQLAlchemy](https://www.sqlalchemy.org/) é uma biblioteca de mapeamento objeto-relacional SQL
- O módulo [shutil](https://docs.python.org/3/library/shutil.html) oferece uma série de operações de alto nível para manipular em arquivos e coleções de arquivos
- O módulo [os](https://docs.python.org/3/library/os.html) apresenta uma maneira portátil de usar as funcionalidades dependente do sistema operacional

In [1]:
import numpy as np
import pandas as pd

import decouple
import sqlalchemy

import shutil
import os

### Leitura e tratamento inicial dos dados de entrada

Para poder realizar a conexão com o banco de dados `SQL`, precisaremos de algumas informações que estão armazenadas no arquivo `.env` e quer serão importadas através da biblioteca `decouple`, como a seguir:

In [2]:
user = decouple.config("db_user_mysql")
host = decouple.config('db_host_mysql')
password = decouple.config('db_password_mysql')
database = decouple.config("db_database_mysql")

Através da biblioteca `sqlalchemy` será criado o objeto [Engine](https://docs.sqlalchemy.org/en/14/core/connections.html#sqlalchemy.engine.Engine) que é baseado na URL do banco de dados.

In [ ]:
connection = sqlalchemy.create_engine("mysql+mysqldb://"+user+":"+password+"@"+host+"/"+database)

Será necessário instalar o pacote cujo nome para instalação no Ubuntu 20.04 é `libmysqlclient-dev` para utilizar o driver `mysqldb`, caso não esteja instalado.

Em seguida, através da biblioteca `pandas` é realizada a importação da tabela 'plan02' utilizando a função `read_sql_table()`, onde o parâmetro `con = connection` é o objeto `engine` criado na célula anterior.

In [3]:
df = pd.read_sql_table('patient01', con = connection)

,0,1,2
nomepaciente,Nicolas Freitas Teste,Gustavo Silveira Teste,Otávio Barros Teste
paciente,1,2,3
datacadastramentopaciente,2019-04-13 17:37:19,2021-04-26 01:54:13,2016-12-13 19:05:57
datanascimentopaciente,1975-03-04 15:28:17,1997-12-09 09:42:22,1983-06-17 22:41:41
conveniopaciente,29.0,34.0,29.0
matriculapaciente,75681864492,57851299323,91162597647
sexopaciente,M,M,F
estadocivilpaciente,S,D,V
corpaciente,Amarelo,Amarelo,Amarelo
documentoidentidadepaciente,734051682,423156809,543278104


### Visualização dos dados

Nas próximas duas células é realizada a visualização inicial dos dados.

Usando o método `head()` do `pandas` com um argumento `4` nele é possível visualizar os primeiros `4` registros do Dataframe.
    
O `.T` significa `Transposição`, desta forma as linhas serão visualizadas como colunas e vice-versa.

In [ ]:
df.head(4).T

O método `info()` do `pandas` apresenta um resumo dos dados no Dataframe, uma informação interessante é o tipo de dado de cada recurso.

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 428 entries, 0 to 427
Data columns (total 40 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   nomepaciente                 428 non-null    object        
 1   paciente                     428 non-null    int64         
 2   datacadastramentopaciente    428 non-null    datetime64[ns]
 3   datanascimentopaciente       428 non-null    datetime64[ns]
 4   conveniopaciente             422 non-null    float64       
 5   matriculapaciente            428 non-null    object        
 6   sexopaciente                 428 non-null    object        
 7   estadocivilpaciente          428 non-null    object        
 8   corpaciente                  428 non-null    object        
 9   documentoidentidadepaciente  428 non-null    object        
 10  cpfpaciente                  428 non-null    object        
 11  graudeinstrucaopaciente      428 non-null    

## Limpeza e tratamento dos dados

A seguir será realizada a limpeza e tratamento dos dados.

### Valores ausentes, valores equivalentes e tratamento inicial

Quando utilizado o método `info()` para ver o resumo dos dados, foi possível ver que muitas colunas tinham muitos dados ausentes, entrentanto, na documentação da iClinic é possível ver que os campos `name` e `birth_date` são obrigatórios e no Dataset anterior as colunas `nomepaciente` e `datanascimentopaciente` são seus equivalentes.

O tratamento de algumas colunas será realizado em seguida, começando com mudança nos nomes das colunas para seus equivalentes no padrão iClinic.

In [5]:
df = df.rename(
    columns = {
        "paciente":"patient_id",
        "nomepaciente": "name",
        "nomeoriginal": "civil_name",
        "datanascimentopaciente":"birth_date", #tratar depois
        "sexopaciente": "gender", #tratar depois
        "cpfpaciente": "cpf", 
        "documentoidentidadepaciente": "rg",
        "telefonespaciente": "mobile_phone", #apenas para inicio do tratamento
        # "email":"email", # como é o mesmo não será modificado
        "naturalidadepaciente": "birth_place",
        "ceppaciente": "zip_code", #tratar depois
        "logradouropaciente": "address", #tratar depois
        "complementopaciente":"complement",
        "bairropaciente": "neighborhood",
        "cidadepaciente": "city",
        "ufpaciente": "state",
        "corpaciente": "ethnicity", #tratar depois
        "estadocivilpaciente": "marital_status", #tratar depois
        "profissaopaciente": "occupation",
        "graudeinstrucaopaciente": "education", #tratar depois
        "observacoespaciente": "observation",
        "indicadoporpaciente": "indication", #tratar depois
        "inativo": "active", #tratar depois
        "pai": "patientrelatedness_father_names",
        "mae": "patientrelatedness_mother_names",
        "foto": "picture_filename", #tratar depois
        "conveniopaciente": "healthinsurance_pack", #tratar depois
    }
)

### Adição de recursos ausentes
Será realizada a adicção de novas colunas que são solicitadas pelo padrão iClinic.

In [6]:
df['social_gender'] = np.nan
df['rg_issuer'] = np.nan
df['home_phone'] = np.nan
df['office_phone'] = np.nan
df['birth_state'] = np.nan 
df['country'] = "BR"
df['religion'] = np.nan
df['responsible'] = np.nan
df['email_secondary'] = np.nan
df['cns'] = np.nan
df['died'] = np.nan
df['death_info'] = np.nan
df['nationality'] = "BR" 
df['indication_observation'] = np.nan 
df['receive_email'] = np.nan
df['tag_names'] = np.nan
df['tag_physician_id'] = np.nan

### Trantamento do recurso 'birth_date'

Como este é um recurso obrigatório e não possui elementos nulos, o único tratamento que aplicaremos será o de formatar de acordo ao padrão solicitado:

In [7]:
df['birth_date'] = pd.to_datetime(df['birth_date'],format='%Y%m%d').dt.date

### Trantamento do recurso 'gender'

Para tratar esta coluna é necessário saber quais as categorias utilizadas para classificar o genero do paciente no Dataset anterior, assim, utilizaremos a seguinte linha de código:

In [ ]:
df['gender'].value_counts()

É possível observar os valores equivalentes no padrão iClinic:

- F : femino : f
- M : masculino : m

Em seguida é aplicado o seguinte comando para fazer a substituição das categorias:

In [ ]:
gender = {
    'M': 'm', 
    'F': 'f'
}

In [8]:
df['gender'] = df['gender'].replace(gender)

### Trantamento do recurso 'mobile_phone'

Como só temos um tipo de telefone armazenado nesse coluna, será necessário apenas formatar o número de telefone armazenado.

A seguir, serão extraídos somente os valores numéricos do recurso 'mobile_phone' e serão armazenados de volta como `string` numa coluna temporaria `rows_t`:

In [ ]:
rows_t = df['mobile_phone'].astype('str').str.extractall('(\d+)').unstack().fillna('').sum(axis=1).astype(int).astype('str')

Finalmente, serão armazenados os numeros de telefone no formato desejado e na sua respectiva coluna:

In [9]:
df['home_phone'] = '(' + rows_t.str[-10:-8] + ')' + rows_t.str[-8:-4] + '-' + rows_t.str[-4:]
df['mobile_phone'] = np.nan

### Trantamento do recurso 'zip_code'

Foi percebido que nem todos os valores armazenado se encontram na formatação desejada, a principal diferença notada foi a falta do hífen dividindo os números, sendo assim será realizado o mapemento dos valores que contém ele a fim de modificar os valores que não o possuem: 

In [10]:
rows_with_dashes = df['zip_code'].str.contains('-')
df['zip_code'] = [df['zip_code'][i] if x == True else df['zip_code'][i][:5]+'-'+df['zip_code'][i][5:] if x == False else np.nan for i, x in enumerate(rows_with_dashes)]

### Trantamento do recurso 'address'

Nesta coluna estão incluidas as informações de outra coluna solicitada no padrão iClinic, i.e., a coluna 'number'.
A seguir, a informação contida no recurso 'address' é dividida com o método `str.split(',')` e será armazenada numa coluna temporária de nome `row_with_adress`:

In [ ]:
row_with_adress = df['address'].str.split(',')

E finalmente, será feita a distribuição da informação de acordo ao seu tipo, como a seguir:

In [11]:
df['number'] = [np.nan if len(x) == 1 else x[1].strip() for x in row_with_adress]
df['address'] = [x[0].strip() for x in row_with_adress]

### Trantamento do recurso 'marital_status'

Para tratar esta coluna é necessário saber quais as categorias utilizadas para classificar o estado civil do paciente no Dataset anterior, assim, utilizaremos a seguinte linha de código:

In [ ]:
df['marital_status'].value_counts()

É possível observar os valores equivalentes no padrão iClinic:

- S: solteiro: si
- C : casado : ma
- O : união estável : st
- V : Viúvo : wi
- D : Separado : se

Em seguida é aplicado o seguinte comando para fazer a substituição das categorias:

In [ ]:
marital_status = {
    'S': 'si',
    'C': 'ma',
    'O': 'st',
    'V': 'wi',
    'D': 'se'
}

In [13]:
df['marital_status'] = df['marital_status'].replace(marital_status)

### Trantamento do recurso 'ethnicity'

Para tratar esta coluna também será necessário saber quais as categorias utilizadas para classificar a cor do paciente no Dataset anterior, assim, analogamente ao realizado no caso anterior:

In [ ]:
df['ethnicity'].value_counts()

Como realizado anteriormente, serão substituidas as categorias pelas suas equivalentes no padrão iClinic, como a seguir:

- Branco : Branca : wh
- Amarelo : Amarela :	ye
- Pardo : Parda : br
- Preto : Negra : bl

Em seguida é aplicado o seguinte comando para fazer a substituição das categorias:

In [ ]:
ethnicity = {
    'Branco': 'wh',
    'Amarelo': 'ye',
    'Pardo': 'br',
    'Preto': 'bl'
}

In [12]:
df['ethnicity'] = df['ethnicity'].replace(ethnicity)

### Trantamento do recurso 'education'

Para tratar esta coluna também será necessário saber quais as categorias utilizadas para classificar a cor do paciente no Dataset anterior, assim, analogamente ao realizado no caso anterior:

In [ ]:
df['education'].value_counts()

Como realizado anteriormente, serão substituidas as categorias pelas suas equivalentes no padrão iClinic, como a seguir:

- Ensino Médio : Ensino Médio : s
- Superior : Ensino Superior :	h
- Pós-graduação : Pós-graduação: p
- Especialização : Pós-graduação: p

Neste caso, foi decidido colocar 'Pós-graduação' e 'Especialização' como 'p', já que não tem nada para específicar se a pós-gradução é lato-sensu ou se não foi informado ter sido mestrado.

Em seguida é aplicado o seguinte comando para fazer a substituição das categorias:

In [ ]:
education = {
    'Ensino Médio': 's',
    'Superior': 'h',
    'Pós-graduação': 'p',
    'Especialização': 'p'
}

In [14]:
df['education'] = df['education'].replace(education)

### Trantamento do recurso 'indication'

Uma vez que todos os valores desta coluna são nulos, iremos colocar o valor 'ot' como sugerido no padrão iClinic.

In [15]:
df['indication']= 'ot'

### Trantamento do recurso 'active'

Aqui será necessário inverter os valores, já que no Dataset anterior o valor armazenado era inactive, logo, apenas substituiremos o valor de `1` pelo `0` e vice-versa:

In [16]:
df['active'] = df['active'].replace({0: 1, 1: 0})

### Trantamento do recurso 'picture_filename'

Para o tratamento deste recurso foi necessário primeiro uma padronização dos valores nulos, assim, foi feita a substituição dos valores 'NONE' e em seguida dos valores 'NaN':

In [17]:
df['picture_filename'] = df['picture_filename'].replace({'NONE': np.nan})
df['picture_filename'] = df['picture_filename'].fillna(np.nan)

Em seguida foram removidos os valores que não representam um endereço apontando para o diretório 'extra/'

In [18]:
rows_begining_with_extra = df['picture_filename'].str.contains(r'^extra')
df['picture_filename'] = [df['picture_filename'][i][6:] if x == True else 'None' for i, x in enumerate(rows_begining_with_extra)]

Após esta filtragem, será feita a movimentação das fotos para um novo diretório:

In [19]:
# Criando o diretório 'pictures/' caso ele não exista
try:
    directory_path = 'desafio-base2-output/picture/'
    os.path.exists(directory_path)
    os.makedirs(directory_path)
    print("Directory created successfully.")

# Caso o diretório já exista
except:
    print("Directory already exist.")

# Movendo todas as fotos indicadas
for picture in df['picture_filename'].values:

    source_path = "desafio-base2/extra/"+picture

    # Caminho de destino

    destination_path = directory_path+picture

    # Copiando as fotos da fonte para seu destino

    try:
        shutil.copy(source_path, destination_path)
        
    # Caso haja algum problema copiando um arquivo já existente no diretório de destino
    except shutil.SameFileError:
        print("Source and destination represents the same file.")

    # Caso haja algum problema de permissão
    except PermissionError:
        print("Permission denied.")

    # Para outro tipo de erros
    except:
        print("Error occurred while copying file.")

print("Pictures copied successfully.")

Directory already exist.
Error occurred while copying file.
Directory already exist.
Error occurred while copying file.
Directory already exist.
Directory already exist.
Error occurred while copying file.
Directory already exist.
Error occurred while copying file.
Directory already exist.
Error occurred while copying file.
Directory already exist.
Error occurred while copying file.
Directory already exist.
Error occurred while copying file.
Directory already exist.
Error occurred while copying file.
Directory already exist.
Directory already exist.
Error occurred while copying file.
Directory already exist.
Directory already exist.
Directory already exist.
Error occurred while copying file.
Directory already exist.
Error occurred while copying file.
Directory already exist.
Error occurred while copying file.
Directory already exist.
Directory already exist.
Error occurred while copying file.
Directory already exist.
Error occurred while copying file.
Directory already exist.
Error occu

Finalmente será modificado o valor da coluna 'picture_filename' para apontar para o novo endereço contendo as fotos 'picture/'.

In [20]:
df['picture_filename'] = ['picture/'+df['picture_filename'][i] if x == True else 'None' for i, x in enumerate(rows_begining_with_extra)]

### Trantamento do recurso 'healthinsurance_pack'

Este recurso precisa de informação externa para ser tratado, pelo que imporatermos o arquivo "planos.csv", mais específicamente as colunas "plan","code" e "name".

In [21]:
df_planos = pd.read_csv('desafio-base2-output/planos.csv', index_col=["plan"], usecols=["plan","code","name"])

plan,1,2,3,4,5,6,7
code,9118087,518103,1377066,3346654,429694,1561209,2031645
name,Unimed,Bradesco,Amil,NotreDame,Porto Seguro,SulAmérica,Allianz


Em seguida é realizada a visualização inicial dos dados.

In [ ]:
df_planos.head(7).T

O autor optou por manter o índice iniciando por 1, considerando 0 como 'None', já que não existe uma definição precisa de por onde o índice deveria começar (por 0 ou 1).

In [ ]:
df['healthinsurance_pack'] = df['healthinsurance_pack'].replace({np.nan: 0})

Foi percebido a existencia de alguns indices fora do conjunto de índices possíves, os que forem maiores que o número de índices dos planos será considerado 0.

In [22]:
tam = df_planos['code'].size
df['healthinsurance_pack'] = [x if x <= df_planos['code'][tam] else 0 for x in df['healthinsurance_pack'] ]
df['healthinsurance_pack'] = df['healthinsurance_pack'].astype(int)

Em seguida é criada uma lista temporária para armazenar os valores da coluna 'healthinsurance_pack':

In [ ]:
index = df['healthinsurance_pack'].values

Também foram criadas mais duas colunas temporárias contendo os respectivos valores para 'healthinsurance_pack_name' e 'healthinsurance_pack_code'.

In [23]:
df['healthinsurance_pack_name'] = [ df_planos['name'][i] if i != 0 else np.nan for i in index ]
df['healthinsurance_pack_code'] = [ df_planos['code'][i].astype(int).astype(str) if i != 0 else np.nan for i in index ]

Finalmente é atualizado o valore da coluna 'healthinsurance_pack' no formato json desejado.


In [24]:
df['healthinsurance_pack'] = 'json::[\\n\\t{\\n\\t\\t"name":"'+df['healthinsurance_pack_name']+'",\\n\\t\\t"code":"'+df['healthinsurance_pack_code']+'"\\n\\t}\\n]'

### Remoção de recursos repetidos ou não necessários

A seguir, serão removidos os recursos que o autor não achou necessários:

In [25]:
df =  df.loc[:, ["patient_id","name","birth_date","gender","cpf","rg","rg_issuer","mobile_phone","home_phone","office_phone","email","email_secondary","birth_place","birth_state","zip_code","address","number","complement","neighborhood","city","state","country","picture_filename","ethnicity","marital_status","religion","occupation","education","responsible","cns","died","death_info","nationality","indication","indication_observation","active","receive_email","observation","healthinsurance_pack","patientrelatedness_mother_names","patientrelatedness_father_names","tag_names","tag_physician_id"]]

### Exportação do arquivo de saída

Como solicitado no desafio, o arquivo de saída será gerado com o conjunto de caracteres `UTF-8`:

In [26]:
df.to_csv('desafio-base2-output/patient.csv',index=False, encoding='utf-8')

Ainda, não foi possível utilizar as seguintes informações:

In [27]:
#  2   datacadastramentopaciente    428 non-null    datetime64[ns]     
#  5   matriculapaciente            428 non-null    object
#  24  prontuario                   428 non-null    object        
#  25  retorno                      428 non-null    datetime64[ns]
#  29  leito                        428 non-null    object        
#  30  alteracao                    428 non-null    datetime64[ns]
#  31  timestamp                    428 non-null    datetime64[ns]
#  34  conjuge                      428 non-null    object        
#  35  filhos                       130 non-null    object        
#  36  empresa                      428 non-null    object  
#  38  cloud                        428 non-null    int64         
#  39  update001                    428 non-null    datetime64[ns]
